Import necessary Library

In [9]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle


import random
import math
import time
from collections import defaultdict  


import warnings
warnings.filterwarnings('ignore')

Information Gain Generator class

In [10]:
class InformationGain:

    
    def calc_entropy(self,column):
        """
        Calculate entropy given a pandas series, list, or numpy array.
        """
        # Compute the counts of each unique value in the column
        counts = np.bincount(column)
        # Divide by the total column length to get a probability
        probabilities = counts / len(column)
        
        # Initialize the entropy to 0
        entropy = 0
        # Loop through the probabilities, and add each one to the total entropy
        for prob in probabilities:
            if prob > 0:
                # use log from math and set base to 2
                entropy += prob * math.log(prob, 2)
        
        return -entropy

    
    def calc_information_gain(self,data, split_name, target_name):
        """
        Calculate information gain given a data set, column to split on, and target
        """
        # Calculate the original entropy
        original_entropy = self.calc_entropy(data[target_name])
        
        #Find the unique values in the column
        values = data[split_name].unique()
        
        
        # Make two subsets of the data, based on the unique values
        left_split = data[data[split_name] == values[0]]
        right_split = data[data[split_name] == values[1]]
        
        # Loop through the splits and calculate the subset entropies
        to_subtract = 0
        for subset in [left_split, right_split]:
            prob = (subset.shape[0] / data.shape[0]) 
            to_subtract += prob * self.calc_entropy(subset[target_name])
        
        # Return information gain
        return original_entropy - to_subtract

          
    def highest_info_gain(self,dataset,columns,target):
        #Intialize an empty dictionary for information gains
        information_gains = {}
        
        #Iterate through each column name in our list
        for col in columns:
          #Find the information gain for the column
          information_gain = self.calc_information_gain(dataset, col, target)
          #Add the information gain to our dictionary using the column name as the ekey                                         
          information_gains[col] = information_gain
        
        #Return the key with the highest value  
        #print(information_gains) 
        list = []
        for i in range(len(columns)) :

              temp = max(information_gains, key=information_gains.get)
              list.append(temp)
              information_gains.pop(temp)

        return list

Processing Telecom Churn Prediction Dataset


In [11]:
def object_to_int(dataframe_series):
    if dataframe_series.dtype=='object':
        dataframe_series = LabelEncoder().fit_transform(dataframe_series)
    return dataframe_series

def featurePriorityForTelcoData():
    df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
    df = df.drop(['customerID'], axis = 1)
    df['TotalCharges'] = pd.to_numeric(df.TotalCharges, errors='coerce')
    df.drop(labels=df[df['tenure'] == 0].index, axis=0, inplace=True)
    df.fillna(df["TotalCharges"].mean())
    df = df.apply(lambda x: object_to_int(x))
    columns =df.columns[:-1]
    infoGain = InformationGain()
    featurePriority = infoGain.highest_info_gain(df,columns,'Churn')
    return featurePriority

def process_telecom_data(feature):
    telco_featurePriority=featurePriorityForTelcoData()

    telco_data = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv") # column sort according to these priority
    telco_data.dtypes

    #treatment of missing value and process the data
    telco_data['TotalCharges'] = pd.to_numeric(telco_data.TotalCharges, errors='coerce')
    telco_data.drop(labels=telco_data[telco_data['tenure'] == 0].index, axis=0, inplace=True)
    telco_data.fillna(telco_data["TotalCharges"].mean())

    feature_used = telco_featurePriority[0:feature]
    feature_used.append('Churn')
    telco_data = telco_data[feature_used]
    #Convertin the predictor variable in a binary numeric variable
    telco_data['Churn'].replace(to_replace='Yes', value=1, inplace=True)
    telco_data['Churn'].replace(to_replace='No',  value=0, inplace=True)
    #Let's convert all the categorical variables into dummy variables
    df_dummies = pd.get_dummies(telco_data)

    # We will use the data frame where we had created dummy variables
    y = df_dummies['Churn'].values
    X = df_dummies.drop(columns = ['Churn'])

    # Scaling all the variables to a range of 0 to 1
    features = X.columns.values
    scaler = MinMaxScaler(feature_range = (0,1))
    scaler.fit(X)
    X = pd.DataFrame(scaler.transform(X))
    X.columns = features
    return train_test_split(X, y, test_size=0.2, random_state=101)

Processing adult dataset

In [12]:
# def object_to_int(dataframe_series):
#     if dataframe_series.dtype=='object':
#         dataframe_series = LabelEncoder().fit_transform(dataframe_series)
#     return dataframe_series

def featurePriorityForAudultData():
      'this function calculate feature priority for audult data'
      missing_values = ["n/a", "na", "--","NA","N/A","?"]
      df = pd.read_csv('adult.data', header=None, na_values = missing_values, skip_blank_lines=True)
      df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation',
                    'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'salary']

      # checking whether it has missing value or not
      df.isnull().sum()
      # no missing value found 
      df = df.apply(lambda x: object_to_int(x))
      columns =df.columns[:-1] #exclude last column which have to be claculated
      infoGain = InformationGain()
      featurePriority = infoGain.highest_info_gain(df,columns,'salary')
      return featurePriority

def process_audult_data(feature):
      'audult data test train spliter'  
      audult_featurePriority=featurePriorityForAudultData()
      missing_values = ["n/a", "na", "--","NA","N/A","?"]
      audult_data =pd.read_csv('adult.data', header=None, na_values = missing_values, skip_blank_lines=True)
      audult_data.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation',
                          'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'salary']
      audult_data.dtypes 
      #checking whether it has missing value or not
      audult_data.isnull().sum()
      #no missing value


      feature_used = audult_featurePriority[0:feature]
      feature_used.append( 'salary')
      audult_data = audult_data[feature_used]
      #Convertin the predictor variable in a binary numeric variable
      audult_data['salary'].replace(to_replace=' >50K', value=1, inplace=True)
      audult_data['salary'].replace(to_replace=' <=50K', value=0, inplace=True)
      audult_data.dtypes
      audult_data.head()
      #Let's convert all the categorical variables into dummy variables
      df_dummies = pd.get_dummies(audult_data)
      df_dummies.head()

      # # We will use the data frame where we had created dummy variables
      y = df_dummies['salary'].values
      X = df_dummies.drop(columns = ['salary'])


      # Scaling all the variables to a range of 0 to 1
      features = X.columns.values
      scaler = MinMaxScaler(feature_range = (0,1))
      scaler.fit(X)
      X = pd.DataFrame(scaler.transform(X))
      X.columns = features
      X.head()
      return train_test_split(X, y, test_size=0.2, random_state=101)



Processing CreditCardFraud Dataset

In [13]:
# def object_to_int(dataframe_series):
#     if dataframe_series.dtype=='object':
#         dataframe_series = LabelEncoder().fit_transform(dataframe_series)
#     return dataframe_series

def featurePriorityForCreditData():
    'this function calculate feature priority for CreditCardFraud data'
    missing_values = ["n/a", "na", "--","NA","N/A","?"]
    df = pd.read_csv('creditcard.csv', na_values = missing_values, skip_blank_lines=True)
    pos_sample = df[df.iloc[:,-1] == 1]
    neg_sample = df[df.iloc[:,-1] == 0].sample(n=20000, random_state=0)
    df = shuffle(pd.concat( (pos_sample,neg_sample), axis=0), random_state=0) 
    df.reset_index(drop=True)
    df.head()
    #checking data type
    df.dtypes
    # checking whether it has missing value or not
    df.isnull().sum()
    # no missing value found 
    df = df.apply(lambda x: object_to_int(x))
    columns =df.columns[:-1] #exclude last column which have to be claculated
    infoGain = InformationGain()
    featurePriority = infoGain.highest_info_gain(df,columns,'Class')
    return featurePriority



def process_credit_data(feature):
    'CreditCardFraud data test train spliter'  
    credit_featurePriority=featurePriorityForCreditData()
    missing_values = ["n/a", "na", "--","NA","N/A","?"]
    credit_data =pd.read_csv('creditcard.csv', na_values = missing_values, skip_blank_lines=True)
    pos_sample = credit_data[credit_data.iloc[:,-1] == 1]
    neg_sample = credit_data[credit_data.iloc[:,-1] == 0].sample(n=20000, random_state=0)
    credit_data = shuffle(pd.concat( (pos_sample,neg_sample), axis=0), random_state=0) 
    credit_data.reset_index(drop=True)
    credit_data.head()
    #checking whether it has missing value or not
    credit_data.isnull().sum()
    #no missing value

    feature_used = credit_featurePriority[0:feature]
    feature_used.append( 'Class')
    credit_data = credit_data[feature_used]

    #Let's convert all the categorical variables into dummy variables
    df_dummies = pd.get_dummies(credit_data)
    df_dummies.head()

    # # We will use the data frame where we had created dummy variables
    y = df_dummies['Class'].values
    X = df_dummies.drop(columns = ['Class'])


    # Scaling all the variables to a range of 0 to 1
    features = X.columns.values
    scaler = MinMaxScaler(feature_range = (0,1))
    scaler.fit(X)
    X = pd.DataFrame(scaler.transform(X))
    X.columns = features
    X.head()
    return train_test_split(X, y, test_size=0.2, random_state=101)





Code for Logistic regression class

In [14]:
class LogitRegression():
          def __init__(self,learning_rate , iterations ):
              self.learning_rate = learning_rate
              self.iterations = iterations

          def fit ( self, X , Y ) :
              self.m , self.n = X.shape
              self.W = np.zeros ( self.n )
              self.b = 0
              self.X = X
              self.Y = Y

              #gradient descent learning
              for i in range( self.iterations) :
                  self.update_weights()
              return self

          def update_weights(self) :
              A = ( np.exp ( ( self.X.dot (self.W ) + self.b ))  - np.exp ( - ( self.X.dot (self.W ) + self.b )) ) / ( np.exp ( ( self.X.dot (self.W ) + self.b ))  + np.exp ( - ( self.X.dot (self.W ) + self.b )) )
              #res = np.identity(len(A)) - (np.dot(A,A.T)) #line1
              tmp = ( A - self.Y.T)
              tmp = np.reshape( tmp , self.m )
              #tmp = np.dot(res,tmp) #line2
              dW = np.dot ( self.X.T , tmp ) / self.m
              db = np.sum(tmp ) / self.m
              self.W = self.W - self.learning_rate * dW
              self.b = self.b - self.learning_rate * db
              return self 

          def predict ( self , X ) :
      
              Z =( np.exp ( ( X.dot (self.W ) + self.b ))  - np.exp ( - ( X.dot (self.W ) + self.b )) ) / ( np.exp ( ( X.dot (self.W ) + self.b ))  + np.exp ( - ( X.dot (self.W ) + self.b )) )
            
              ans = np.where(Z> 0.5 , 1 , 0)
             
              return ans

Code for Adaboost class

In [15]:

class Adaboost:
    
    def __init__(self, numOfHypothesis):
        self.hypoNum = numOfHypothesis
        self.hypoList = [] 
        self.hypoWeight = [1] * numOfHypothesis

    def normalize(self, lst):
        lst = np.array(lst)
        sum = np.sum(lst)
        return lst/sum

    def getResampleData(self):
        chosenIdxs = random.choices( population= list( range( len(self.dataset) ) ), weights=self.sampleWeight, k=len(self.dataset) )
        return self.dataset[chosenIdxs]

    def fit(self, x_train, y_train):
        x_train = np.array(x_train, dtype="float")
        y_train = np.array(y_train,  dtype="float")
        self.dataset = np.column_stack((x_train, y_train))
        self.sampleWeight = [1/len(self.dataset)] * len(self.dataset)

        k = 0
        while k < self.hypoNum:
            #dtc = DecisionTree(1)
            dtc = LogitRegression( learning_rate = 0.1 , iterations = 1000)
            data = self.getResampleData()

            # dtc = DecisionTreeClassifier(random_state=0, max_depth=1)
            # data = self.dataset
            tmpHypoResult = self.hypothesisResult( dtc, data )
            #tmpHypoResult = dtc.fit()
            error = self.getError(tmpHypoResult, y_train)

            if error > 0.5:
                continue
            for j in range(len(y_train)):
                if tmpHypoResult[j] == y_train[j]:
                    self.sampleWeight[j] = self.sampleWeight[j] * error/(1 - error)
            
            self.sampleWeight = self.normalize(self.sampleWeight)
            self.hypoList.append(dtc)
            self.hypoWeight[k] = math.log2( (1-error)/ error )
            k = k + 1




    def getError(self, y_pred, y_test):
        error = 0
        for i in range(len(y_pred)):
            if y_pred[i] != y_test[i]:
                error = error + self.sampleWeight[i]
        return error

    def hypothesisResult(self, dtc, resampleData ):
        dtc.fit(resampleData[:,:-1], resampleData[:,-1] )

        return dtc.predict(self.dataset[:,:-1])
 

    def predict(self, x_test):
        y_predLst = []
        y_test = []
        for hypo in self.hypoList:
            y_test.append( hypo.predict(x_test) ) 
            
        y_test = np.array(y_test)
        for i in range(len(x_test)):
            y_pred = y_test[:,i]
            voting = defaultdict(int)
            for k in range( self.hypoNum ):
                voting[ y_pred[k] ] += self.hypoWeight[k]
            y_predLst.append( max(voting, key=voting.get) ) 
        
        return y_predLst

Execution

In [ ]:



def main():

    # input for telecom data
    #X_train, X_test, y_train, y_test = process_telecom_data(18)
    # input for audult data
    #X_train, X_test, y_train, y_test = process_audult_data(8)
    # input for credit data
    X_train, X_test, y_train, y_test = process_credit_data(8)

    for stump in [5,10,15,20]:
      boost = Adaboost(stump)

      start_time = time.time()
      print('-- Data fitting start')

      # boost.fit(x_train, y_train)
      boost.fit(X_train, y_train)

      print('Data fitting finished')
      print('Time taken ----', time.time() - start_time)

      print('Data prediction start')
      start_time = time.time()

      y_pred = boost.predict(X_test)
      y_train_pred = boost.predict(X_train)
      print('Data fitting finished')
      print('Time taken ----', time.time() - start_time)

      print('No of stumps ',stump)
      print('Training set performance')
      print(metrics.accuracy_score(y_train,y_train_pred) * 100)

      print('Testing set performance')
      print(metrics.accuracy_score(y_test,y_pred) * 100)
      print('Data prediction finished')
      print('Time taken ----', time.time() - start_time)
      print('------------------------------------------')
   
main()


-- Data fitting start
